In [24]:
import os ## here we will load the env file so you dont need to hardcode your api key
from dotenv import load_dotenv
import streamlit as st ## using streamlit for forntend or web ui
load_dotenv()
groq_api_key=os.getenv("GROQ_API_KEY")## it will fetch the api key and print it on a streamlit for testing later you can remove
groq_api_key



'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'

In [25]:
from langchain_groq import ChatGroq    ##creating LLM model and i am using grocks gemma2-9b-it model and chatgroq is the wrapper and ican interact with this model using langchain
model=ChatGroq(model="Gemma2-9b-IT",groq_api_key=groq_api_key)
model


ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000024540237110>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000024540309310>, model_name='Gemma2-9b-IT', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [26]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi , My name is Ashwini and i am a Cheif AI Engineer")])

## now ere i am sending a simple message to the LLm model and it will reply like a chatboat

AIMessage(content="Hello Ashwini, it's nice to meet you!\n\nThat's a fascinating title. As a large language model, I'm always interested in learning more about the work of AI engineers. What kind of projects are you working on these days?\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 55, 'prompt_tokens': 24, 'total_tokens': 79, 'completion_time': 0.1, 'prompt_time': 0.002211295, 'queue_time': 0.23448644899999999, 'total_time': 0.102211295}, 'model_name': 'Gemma2-9b-IT', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-01367618-96b4-45bf-80d7-52f79b13ae5c-0', usage_metadata={'input_tokens': 24, 'output_tokens': 55, 'total_tokens': 79})

In [27]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi , My name is Ashwini and i am a Cheif AI Engineer"),
        AIMessage(content="It's nice to meet you, Ashwini! \n\nThat's a fascinating title. As a Chief AI Engineer, I imagine you're involved in some cutting-edge work. What are you currently working on that you're most excited about?  \n\nI'm always eager to learn more about the applications of AI.\n"),
        HumanMessage(content="Hey What's my name and what do i do?")
                  
    ]
)
''' here i am sending so many messages to the model this is the conversation between human and ai and this will allow model to understand chat context not just a latest message'''

' here i am sending so many messages to the model this is the conversation between human and ai and this will allow model to understand chat context not just a latest message'

In [28]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]  

with_message_history=RunnableWithMessageHistory(model,get_session_history)  
    
''' now in this code i am setting up the chat memory so my chatboat can remember what i said in the earlier chat session'''
''''''

''

In [29]:
config = {"configurable": {"session_id": "chat1"}}

In [30]:
response = with_message_history.invoke(
    [HumanMessage(content="Hi, My Name is Ashwini and I am a Chief AI Engineer")],
    config=config
)

In [31]:
response.content

"Hello Ashwini, it's nice to meet you! \n\nBeing a Chief AI Engineer is a fascinating role. What kind of projects are you currently working on?  I'm always eager to learn more about the cutting edge of AI development.\n"

In [32]:
response1 = with_message_history.invoke(
    [HumanMessage(content="what's my Name?")],
    config=config
)
response1.content

'Your name is Ashwini.  \n\nI remember that from our introduction! 😊  Is there anything else I can help you with?  \n'

In [33]:
config1 = {"configurable": {"session_id": "chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="What's My Name?")],
    config=config1
)
response.content

"As an AI, I don't have access to any personal information about you, including your name. Could you tell me your name? 😊\n"

In [34]:
response = with_message_history.invoke(
    [HumanMessage(content="Hey My Name is Sapna?")],
    config=config1
)
response.content

"Hi Sapna, it's nice to meet you! 👋  \n\nHow can I help you today? 😄  \n"

In [35]:


config = {"configurable": {"session_id": "chat1"}}
response=with_message_history.invoke(
    [HumanMessage(content="What's My Name?")],
    config=config
)
response.content

"Your name is Ashwini.  I'm happy to remind you! 😄  \n\nIs there anything else I can help you with today? \n"

In [36]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt =  ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Your are a helpful assistant. Answer all questions to the best of your ability in {language}"
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain=prompt|model
response=chain.invoke({"messages":[HumanMessage(content="Hi My name is Ashwini")],"language":"Hindi"})


with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)


In [37]:
config = {"configurable": {"session_id": "chat5"}}
repsonse=with_message_history.invoke(
    {'messages': [HumanMessage(content="Hi,I am Ashwini")],"language":"Hindi"},
    config=config
)
repsonse.content

repsonse1=with_message_history.invoke(
    {'messages': [HumanMessage(content="What's my name")],"language":"English"},
    config=config
)
repsonse1.content

"Your name is Ashwini.  \n\nIt's nice to meet you, Ashwini! 👋  Is there anything I can help you with today?\n"

In [38]:
from langchain_core.messages import SystemMessage,trim_messages
trimmer=trim_messages(
    max_tokens=70,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"    
)
messages = [
    SystemMessage(content="you are a good assistant"),
    HumanMessage(content="hi! i am bob "),
    AIMessage(content="hi!"),
    HumanMessage(content="I like choclate ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="Whats 2+2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem"),
    HumanMessage(content="Having fun"),
    AIMessage(content="yes"),
]
trimmer.invoke(messages)


[SystemMessage(content='you are a good assistant', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='hi! i am bob ', additional_kwargs={}, response_metadata={}),
 AIMessage(content='hi!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like choclate ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Whats 2+2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Having fun', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes', additional_kwargs={}, response_metadata={})]

In [39]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages") |trimmer)
    | prompt
    | model
    
)

response1=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What ice cream do i like")],
    "language":"English"
    }
)
response1.content
 

'You said you like chocolate ice cream!  🍦🍫  \n'

In [40]:
from langchain_core.documents import Document

documents = [
    Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Goldfish are popular pets for beginners, requiring relatively simple care.",
        metadata={"source": "fish-pets-doc"},
    ),
    Document(
        page_content="Parrots are intelligent birds capable of mimicking human speech.",
        metadata={"source": "bird-pets-doc"},
    ),
    Document(
        page_content="Rabbits are social animals that need plenty of space to hop around.",
        metadata={"source": "mammal-pets-doc"},
    ),
]


In [41]:
documents

[Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(metadata={'source': 'fish-pets-doc'}, page_content='Goldfish are popular pets for beginners, requiring relatively simple care.'),
 Document(metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around.')]

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()
from langchain_groq import ChatGroq
groq_api_key=os.getenv("GROQ_API_KEY")

os.environ["HF_TOKEN"]=os.getenv("HF_TOKEN")

llm=ChatGroq(groq_api_key=groq_api_key.model-"Llama3-8b-8192")
llm


